In [0]:
def mount_drive():
  from google.colab import drive
  import os
  drive.mount('gdrive')
  os.chdir('gdrive/My Drive/movie plot')

In [0]:
def get_embedding_dictionary():
  import numpy as np
  return np.load("embedding dictionary.npy",allow_pickle=True).item()

In [0]:
def get_tokenizer(corpus,num_words):
  from tensorflow.keras.preprocessing.text import Tokenizer
  tokenizer=Tokenizer(num_words=num_words, oov_token='<OOV>')
  tokenizer.fit_on_texts(train_plots)
  return tokenizer

In [0]:
def get_train():
  import pandas as pd
  train=pd.read_csv("train.csv")
  train_plots=train["plots"].to_numpy()
  train_labels=train["labels"].to_numpy()
  return train_plots,train_labels

In [0]:
def get_test():
  import pandas as pd
  test=pd.read_csv("test.csv")
  test_plots=test["plots"].to_numpy()
  test_labels=test["labels"].to_numpy()
  return test_plots,test_labels

In [0]:
def get_padding_sentence(corpus,tokenizer,maxlen):
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  seq=tokenizer.texts_to_sequences(corpus)
  seq = pad_sequences(seq, padding='post',maxlen=maxlen)
  return seq

In [0]:
def get_sentence_embedding(sentence_batch,tokenizer):
  no_embedding=0
  with_embedding=0
  import numpy as np
  word2embedding=get_embedding_dictionary()
  index2word = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
  bound = np.sqrt(6.0) / np.sqrt(10000) 
  embeddings=[]
  for sentence in sentence_batch:
    vectors=[]
    for index in sentence:
      if index!=0:
        word=index2word[index]
      else:
        word=None
      embedding=None
      try:
        embedding=word2embedding[word]
        with_embedding+=1
      except:
        no_embedding+=1
        pass
      if embedding is not None:
        vectors.append(embedding)
      else:
        vectors.append(np.random.uniform(-bound, bound, 300))
    embeddings.append(vectors)
  print("%d words have embedding, while"%with_embedding,"%d words do not"%no_embedding)
  return np.array(embeddings)

In [0]:
def get_sentence_embedding_(sentence_batch,tokenizer):
  no_embedding=0
  with_embedding=0
  import numpy as npimport sys
sys.path.append('/content/gdrive/mypythondirectory')
  #word2embedding=get_embedding_dictionary()
  from gensim import models

  model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True) 
  index2word = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
  bound = np.sqrt(6.0) / np.sqrt(10000) 
  embeddings=[]
  for sentence in sentence_batch:
    vectors=[]
    for index in sentence:
      if index!=0:
        word=index2word[index]
      else:
        word=None
      embedding=None
      try:
        embedding=model.word_vec(word)
        with_embedding+=1
      except:
        no_embedding+=1
        pass
      if embedding is not None:
        vectors.append(embedding)
      else:
        vectors.append(np.random.uniform(-bound, bound, 300))
    embeddings.append(vectors)
  print("%d words have embedding, while"%with_embedding,"%d words do not"%no_embedding)
  return np.array(embeddings)

In [0]:
mount_drive()
train_plots,train_labels=get_train()
tokenizer=get_tokenizer(train_plots,20000)
#train_plots=get_padding_sentence(train_plots,tokenizer,100)

In [0]:
#e=get_sentence_embedding_(train_plots,tokenizer)
#print(e.shape)

In [0]:
#import numpy as np
#np.save("train plots with embedding(first 100 grams).npy",e)

In [0]:
#ee=np.load("train plots with embedding(first 100 grams).npy")
#print(ee.shape)

In [0]:
test_plots,test_labels=get_test()
test_plots=get_padding_sentence(test_plots,tokenizer,100)
e=get_sentence_embedding_(test_plots,tokenizer)
print(e.shape)

In [0]:
import numpy as np
np.save("test plots with embedding(first 100 grams).npy",e)